In [1]:
import pandas as pd
import numpy as np

In [2]:
basics = pd.read_csv('Data/basics.tsv', sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357313 entries, 0 to 9357312
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 642.5+ MB


In [4]:
ratings = pd.read_csv('Data/ratings.tsv', sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1733
3,tt0000004,5.6,174
4,tt0000005,6.2,2544


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245557 entries, 0 to 1245556
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1245557 non-null  object 
 1   averageRating  1245557 non-null  float64
 2   numVotes       1245557 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.5+ MB


In [6]:
akas = pd.read_csv('Data/akas.tsv', sep='\t', low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [7]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33762801 entries, 0 to 33762800
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


In [8]:
#replace \N with traditional NaN value
basics.replace({'\\N':np.nan},inplace=True)
ratings.replace({'\\N':np.nan},inplace=True)
akas.replace({'\\N':np.nan},inplace=True)

In [9]:
basics['runtimeMinutes'].value_counts(dropna=False,sort=True)

NaN     6800352
30       145454
22       114499
60       108299
44        70365
         ...   
665           1
3087          1
583           1
503           1
2088          1
Name: runtimeMinutes, Length: 878, dtype: int64

In [10]:
#drop movies with unknown runtime
basics = basics[basics['runtimeMinutes'].notna()]

In [11]:
#drop movies with unknown genre
basics = basics[basics['genres'].notna()]

In [12]:
#only keep full-length movies
basics = basics[basics.titleType =='movie']

In [13]:
#exclude documentaries
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [14]:
#drop null startYears so we can typecast to int and apply filters
basics = basics[basics['startYear'].notna()]
basics['startYear'].astype(int)

8          1894
570        1906
587        1907
672        1908
930        1909
           ... 
9356985    2019
9356994    2020
9357033    2020
9357078    2020
9357162    2019
Name: startYear, Length: 278150, dtype: int32

In [15]:
#keep movies made in 2000 - 2021
basics = basics[(basics.startYear > '1999') & (basics.startYear < '2022')]

In [16]:
#only keep movies released in the US
akas = akas.loc[akas['region']=="US"]

In [17]:
# Filter the basics table down to only include the US by using the filter akas dataframe
us_titles =basics['tconst'].isin(akas['titleId'])
basics = basics[us_titles]

us_titles2 =ratings['tconst'].isin(akas['titleId'])
ratings = ratings[us_titles2]

In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79861 entries, 34793 to 9357078
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          79861 non-null  object
 1   titleType       79861 non-null  object
 2   primaryTitle    79861 non-null  object
 3   originalTitle   79861 non-null  object
 4   isAdult         79861 non-null  object
 5   startYear       79861 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  79861 non-null  object
 8   genres          79861 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [19]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365697 entries, 5 to 33762545
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1365697 non-null  object
 1   ordering         1365697 non-null  int64 
 2   title            1365697 non-null  object
 3   region           1365697 non-null  object
 4   language         3716 non-null     object
 5   types            965971 non-null   object
 6   attributes       45278 non-null    object
 7   isOriginalTitle  1364322 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.8+ MB


In [20]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474085 entries, 0 to 1245553
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         474085 non-null  object 
 1   averageRating  474085 non-null  float64
 2   numVotes       474085 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.5+ MB


In [24]:
## Save current dataframe to file.
basics.to_csv("Data/basics.csv.gz",compression='gzip',index=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34793,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61096,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67642,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86772,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93908,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [25]:
## Save current dataframe to file.
akas.to_csv("Data/akas.csv.gz",compression='gzip',index=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [26]:
## Save current dataframe to file.
ratings.to_csv("Data/ratings.csv.gz",compression='gzip',index=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
4,tt0000005,6.2,2544
5,tt0000006,5.1,175
6,tt0000007,5.4,797
